In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#We import tensorflow library for deep neural network.
import seaborn as sns
import tensorflow as tf

In [ ]:
#we prepare our directories
mother_directory = os.getcwd()
data_directory = "../input/early-diabetes-classification/diabetes_data.csv"

In [ ]:
def categorical_to_binary(data,categorical_dict):
    sütun_list = []
    for sütun in data.columns:
        if type(data[sütun].iloc[0]) is str:
            çeşitler = data[sütun].unique()
            sütun_list.append(sütun)
            for kaş in range(len(çeşitler)-1):
                categorical_dict[çeşitler[kaş]] = []
                for j in range(len(data[sütun])):
                    if data[sütun].iloc[j] == çeşitler[kaş]:
                        categorical_dict[çeşitler[kaş]].append(1)
                    else:
                        categorical_dict[çeşitler[kaş]].append(0)

    return sütun_list

In [ ]:
#Here we grab our data into a dataframe.
data = pd.read_csv(data_directory,delimiter=";")

In [ ]:
data.head()


In [ ]:
#we turn categorical into one hot encoded columns
categorical_dict = {}
liste_sütun = categorical_to_binary(data,categorical_dict)

In [ ]:
#we change the data into one hot encoded
for name in categorical_dict.keys():
    data[name] = categorical_dict[name]
for k in liste_sütun:
    data = data.drop(k,axis=1)

In [ ]:
#Heat map to look at the correlations between the data categories
tc = data.corr()
sns.heatmap(tc,annot = False,cmap="coolwarm")

In [ ]:
data.info()


In [ ]:
#Here we prepare data adn train it on the neural network we defined.
malum = data["obesity"].copy()
data = data.drop("obesity",axis=1)
X = data.iloc[:,:].values
y = malum.values

#Here split our data into train_test datasets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.15,random_state = 10)
print(X_train)
print(y_train)
yedek = y_test.copy()

#here we standardize the data into the same interval so that a category wouldnt overrepresent in the decision process.
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_train = sc_x.fit_transform(X_train)
X_test = sc_x.transform(X_test)

#We define our neural network and compile it
classifier = tf.keras.models.Sequential()
classifier.add(tf.keras.layers.Dense(units = 128, activation = "relu"))
classifier.add(tf.keras.layers.Dropout(0.2))
classifier.add(tf.keras.layers.Dense(units = 32, activation = "relu"))
classifier.add(tf.keras.layers.Dropout(0.1))
classifier.add(tf.keras.layers.Dense(units = 1,activation = "sigmoid"))
classifier.compile(optimizer = "adam", loss = "binary_crossentropy" , metrics=["accuracy"])

#Here we train our model.
classifier.fit(X_train,y_train,epochs = 100,validation_data = (X_test,y_test))
#This the inference phase.We try our model on test data.
y_pred = classifier.predict(X_test)

y_pred = (y_pred > 0.5)
from sklearn.metrics import confusion_matrix,accuracy_score
#With this confusion matrix 
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

In [ ]:
#This the inference phase.We try our model on test data.
y_pred = classifier.predict(X_test)

y_pred = (y_pred > 0.5)
from sklearn.metrics import confusion_matrix,accuracy_score
#With this confusion matrix 
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

Reference published Article : https://doi.org/10.1007/978-981-13-8798-2_12
